In [1]:
import pandas as pd
import numpy as np
import random as rd 
import pickle as pk

from Judge.Score import *   # 评价指标

In [2]:
df_ft = pd.read_csv('./Datasets/train_feature.csv')
df_lb = pd.read_csv('./Datasets/train_label.csv') 

# df_origin = pd.concat([df_ft_origin, df_lb_origin],axis=1)


## 数据预处理
首先进行数据的预处理。

In [3]:
# # print(df_ft_origin.head())
# print(df_ft_origin['feature_0'])
# print(type(df_ft_origin['feature_0']))
# print(df_ft_origin.mean())
# print(df_ft_origin.median())
# print(df_ft_origin.var())



In [4]:
from Preprocessing import Tools as M_TOOL
from Preprocessing import PCA as M_PCA

from itertools import chain as ch


df = pd.concat([df_ft, df_lb], axis=1)
df.fillna(df.median(), inplace=True)

# 删除噪声
df = M_TOOL.Drop_noise(df, if_debug=False)

# 数据归一化
df = M_TOOL.Normalization(df, if_debug=False)


# PCA_model = M_PCA.PCA(df.values)
# df = pd.DataFrame(PCA_model.reduce_dimension())
# print(df.head())
# corr = np.array(df.corr())
# print(df.corr())


# def flat(nest,cond_func=lambda r:type(r)==ast.Num,get_func=lambda r:r.n):
#     cd = json.dumps(nest)
#     t = ast.parse(cd)
#     g = ast.walk(t)
#     arr = list(g)
#     arr = list(filter(cond_func,arr))
#     arr = list(map(get_func,arr))
#     return(arr)

# 作者：navegador
# 链接：https://www.zhihu.com/question/356442472/answer/1585424932
# 来源：知乎
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。



# corr_list = list(ch.from_iterable(corr))

# corr_list = list(set(corr_list).difference(set([corr_list[i] for i in range(len(corr_list)-1,-1,-1) if corr_list[i] == 1.0])))

# corr_list.sort()
# # print(corr_list)

# max_list = corr_list[-5:]
# print(max_list)


    
# sub_df_1 = df[df['label'] == 0]






# df.drop_duplicates(inplace=True)
# print(df.shape)
# print(sub_df_1.head())
# print(sub_df_1.corr())

print(df.shape)
X_train, y_train, X_test, Y_test = M_TOOL.random_Split_data(df)


(5195, 121)


In [5]:
# from sklearn.manifold import *

# m = LocallyLinearEmbedding(n_components=80)
# m.fit(X_train, y_train)
# X_train = m.transform(X_train)
# X_test = m.transform(X_test)



## 模型训练与预测

接下来，我们将分别使用线性回归模型、决策树模型、神经网络模型、支持向量机以及 XGBoost 等分类模型对数据集进行训练，并在验证集上进行预测。

In [6]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression

#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
print(X_train.shape)
X_new = RFE(estimator=LinearRegression(), n_features_to_select=50).fit_transform(X_train, y_train)
print(X_new.shape)
drop_index = M_TOOL.Find_drop_feature(X_train, X_new)
# print(drop_index)

X_train = np.delete(X_train, drop_index, axis=1)
X_test = np.delete(X_test, drop_index, axis=1)


M_TOOL.Find_useless_feature(df)


(3896, 120)
(3896, 50)
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.423077   0.621537   0.064280   0.333333   0.384615   0.696370   
1   0.365385   0.210505   0.590200   0.777778   0.153846   0.465347   
2   0.576923   0.487735   0.173248   0.555556   0.076923   0.828383   
3   0.461538   0.556613   0.004739   0.555556   0.230769   0.613861   
4   0.442308   0.358241   0.709728   0.666667   0.230769   0.792079   

   feature_6  feature_7  feature_8  feature_9  ...  feature_111  feature_112  \
0   0.367347        0.5        0.1   0.040544  ...     0.098502     0.340426   
1   0.428571        0.4        0.0   0.049125  ...     0.049627     0.723404   
2   0.408163        0.3        0.0   0.015803  ...     0.063077     0.723404   
3   0.244898        0.5        0.0   0.044818  ...     0.099858     0.680851   
4   0.612245        0.3        0.0   0.047807  ...     0.090963     0.553191   

   feature_113  feature_114  feature_115  feature_116  feature_117  \

In [7]:
from Methods import LinearReg as M_LR
# 线性回归模型

model_liner = M_LR.LinearRegression(X_train, y_train)
model_liner.fit()
# model_liner.save()
# model_liner = pk.load(open("./model/Model_LinearReg_2022_12_24_21_07_58.dat", "rb"))    # 使用已保存的模型

pre = model_liner.predict(X_test)



acc = Accuracy(pre, Y_test)
print("线性回归模型分类准确度为 {}".format(acc))


# 与 Sklearn 比较
from sklearn.linear_model import LinearRegression as LR  
model_liner_skl = LR().fit(X_train, y_train)
pre_skl = model_liner_skl.predict(X_test)

acc = Accuracy(pre_skl, Y_test)
print("SKLearn 分类准确度为 {}".format(acc))


线性回归模型分类准确度为 0.23787528868360275
SKLearn 分类准确度为 0.2424942263279446


In [8]:
# # 决策树模型

# from Methods import DecTree as M_DTC
# parameters = {
#     "criterion": "gini",    # 选择特征的标准，分为 "gini" 和 "entropy"
#     "splitter": "best",        # 特征划分标准，分为 "best" 和 "random"
#     "max_depth": 8,
#     "min_samples_split": 2,    # 叶子最小样本数
#     "min_samples_leaf": 1,     # 划分最小样本数
#     "max_leaf_nodes": 10000,    # 最大叶节点个数
#     "min_impurity_decrease": 0.0, # 最小划分不纯度减少量
#     "if_silent": 1
# }

# model_dtc = M_DTC.DecisionTree(X_train, y_train)
# model_dtc.fit(parameters)

# pre = model_dtc.predict(X_test)
# pre = np.array(pre).reshape(-1, 1)


# acc_dtc = Accuracy(pre, Y_test)
# print("决策树分类准确度为 {}".format(acc_dtc))


In [9]:
# from Methods import RegTree as M_RT
# # 回归树模型

# m, n = y_train.shape 
# y_t = np.zeros((m, n))
# model_regtree = M_RT.RegressionTree(X_train, y_train, y_t)
# default_parameters = {
#     "lamda": 2,             # Hyperparameters
#     "gamma": 1e-6,          # Hyperparameters
#     "gain_delta": 0,        # The minimum gain
#     "max_depth": 3,
#     "max_leaves": 100,
#     "max_nodes": 1000,
#     "min_samples": 10,      # Minimum number of samples on a leaf
#     "min_feature_dif": 5,   # The minimum number of different value for current feature
#     "if_silent": 0
# }
# model_regtree.fit(default_parameters)
# pre = model_regtree.predict(X_test)

# acc = Accuracy(pre, Y_test)
# print("回归树模型分类准确度为 {}".format(acc))


In [10]:
# from Methods import SVM as M_SVM
# # 支持向量机模型

# model_SVM = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM.fit(max_times=500, ifsilent=True)
# pre = model_SVM.predict(X_test)

# acc = Accuracy(pre, Y_test)
# print("SVM 模型分类准确度为 {}".format(acc))


# # 与 SKLearn 比较
# from sklearn import svm 
# model = svm.SVC(kernel='linear')
# model.fit(X_train, y_train.flatten())
# pre_skl = model.predict(X_test).reshape(-1, 1)

# acc = Accuracy(pre_skl, Y_test)
# print("SKLearn 分类准确度为 {}".format(acc))

In [11]:
# from Methods import NeuralNet as M_NN
# # 神经网络模型

# model_NN = M_NN.NeuralNetwork(X_train, y_train.flatten())
# model_NN.fit()
# pre = model_liner.predict(X_test)

# acc_1 = Accuracy(pre, Y_test)
# print("线性回归模型分类准确度为 {}".format(acc_1))

In [12]:
from Methods import XGBoost as M_XGB
# XGBoost 模型

model_XGB = M_XGB.XGBoost(X_train, y_train)

model_XGB.fit(T=100, max_depth=8)
pre = model_XGB.predict(X_test)
pre = np.array(pre).reshape(-1, 1)

acc_xgb = Accuracy(pre, Y_test)
print("XGBoost 模型分类准确度为 {}".format(acc_xgb))


XGBoost 模型分类准确度为 0.2517321016166282
